In [ ]:
# =======================================================
# Primero editar si se quieren cortes en sfr o stellarmass!
# y seleccionar el corte deseado (ver archivo de densidades)
# ======================================================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os
from cuts import density_data

rcParams['font.family'] = 'serif'

In [ ]:
path = "/home/esteban/Escritorio/Practica/Data/G13_millennium"  # Guo2013    (MPA)
#path = "/home/esteban/Escritorio/Practica/Data/G14_millennium" # Gonzalez2014  (Durham)
#path = "/home/esteban/Escritorio/Practica/Data/L16_millennium" # Lacey2016     (Durham)
os.chdir( path )

In [ ]:
sfr = np.load('sfr.npy')
stellarmass = np.load('stellarmass.npy')
#fofid = np.load('fofid.npy')
centralmvir = np.log10(np.load('centralmvir.npy') + 1e-10) + 10 # mdhalo in MPA database
Type = np.load('type.npy')

# Density samples (Only 1 run!) Done!
#density_data(sfr, stellarmass)

In [ ]:
log_centralmvir = centralmvir
logm_min = 10
logm_max = 15.
NBIN = 30

mass_labels = ((log_centralmvir - logm_min)/(logm_max - logm_min) * NBIN).astype(int)
bin_width = (logm_max - logm_min)/NBIN
bins = np.array([10 + bin_width*i for i in range(NBIN)]) + bin_width *.5

In [ ]:
%%time
# Density selection
# =================
#   Fila    log(rho)
#    15      -2.5
#    10      -3.0
#     5      -3.5
# =================
log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)
rho, mass_cut, sfr_cut = log_rho[5], mass_cuts[5], sfr_cuts[5]

# To make HODs
HOD = np.zeros(NBIN)
Centrals = np.zeros(NBIN)
Sats = np.zeros(NBIN)


for i in range(NBIN):
    
    idx = np.where(mass_labels == i)[0]
    Halos = np.where(Type[idx] == 0)[0] 
    
    mask_HOD = sfr[idx] > sfr_cut # Msun/year
    HOD_ID = np.where(mask_HOD)[0]
    
    mask_Cent = np.where(Type[idx][HOD_ID] == 0)[0]
    mask_Sats = np.where(Type[idx][HOD_ID] != 0)[0]
    
    Len_Full = len(HOD_ID)
    Len_Cent = len(mask_Cent)
    Len_Sats = len(mask_Sats)
    Len_Haloes = float(len(Halos)) + 1e-10
    
    HOD[i]      = Len_Full / Len_Haloes 
    Centrals[i] = Len_Cent / Len_Haloes
    Sats[i]     = Len_Sats / Len_Haloes


In [ ]:
#np.save("HOD_sfr_01.txt", np.array([bins, HOD, Centrals, Sats]).T)

In [ ]:
f = plt.figure()
line = np.zeros(NBIN)

path = "/home/esteban/Escritorio/Practica/Plots/G13_millennium" # Guo2013 
#path = "/home/esteban/Escritorio/Practica/Plots/G14_millennium" # Gonzalez2014 
#path = "/home/esteban/Escritorio/Practica/Plots/L16_millennium" # Lacey2016 
os.chdir( path )

plt.plot(bins, np.log10(HOD + 1e-10), 'k.', label = 'All')
plt.plot(bins, np.log10(Centrals + 1e-10), 'r--', label = 'Centrals')
plt.plot(bins, np.log10(Sats + 1e-10), 'b--', label = 'Sats')
plt.plot(bins, line, 'k--', lw = .5 )
plt.title(r"$n = 10^{%.1f} /h^{-3} Mpc^3$" %rho)
plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$")
plt.ylabel(r"$log(N)$")
plt.ylim(-3,3)
plt.xlim(10,15)
plt.minorticks_on()
plt.legend(loc='upper left', frameon = False, prop={'size':10})
f.savefig('hod_04.pdf')